# Conditional MC of Heston Model based on QE Scheme

In [1]:
import numpy as np
import heston_cmc_qe as heston
import time
import pyfeng as pf
import matplotlib.pyplot as plt
from tqdm import tqdm

## Andersen's (2008) and Van's (2010) examples, with multiple strikes and single spot

In [2]:
# Examples
# Andersen (2008)
strike = [100.0, 140.0, 70.0]
forward = 100
delta = [1, 1/2, 1/4, 1/8, 1/16, 1/32]
case = np.zeros([3, 7])
#case[i]=[vov, kappa, rho, texp, theta,  sigma,     r]
case[0] = [1,   0.5, -0.9, 10, 0.04, np.sqrt(0.04), 0]
case[1] = [0.9, 0.3, -0.5, 15, 0.04, np.sqrt(0.04), 0]
case[2] = [1,   1,   -0.3, 5,  0.09, np.sqrt(0.09), 0]

In [3]:
# Van (2010)
strike = [100.0, 140.0, 60.0]
forward = 100
delta = [1, 1/2, 1/4, 1/8, 1/16, 1/32]
case = np.zeros([3, 7])
#case[i]=[vov, kappa, rho, texp, theta,   sigma,    r]
case[0] = [1,   0.5, -0.9, 10, 0.04, np.sqrt(0.04), 0]
case[1] = [1,   1,   -0.3, 5,  0.09, np.sqrt(0.09), 0.05]
case[2] = [0.9, 0.3, -0.5, 15, 0.04, np.sqrt(0.04), 0]
ref_price = np.array([[13.085, 0.296, 44.330], [33.597, 18.157, 56.575], [16.649, 5.138, 45.287]])

In [4]:
# compute conditional mc price and bias
# for cases with multiple strike and single forward
price_cmc = np.zeros([len(delta), len(strike)])
bias_cmc = np.zeros([len(delta), len(strike)])
for i in range(case.shape[0]):
    vov, kappa, rho, texp, theta, sigma, r = case[i]
    
    start = time.time()
    heston_cmc_qe = heston.HestonCondMcQE(vov=vov, kappa=kappa, rho=rho, theta=theta)
    
    for d in range(len(delta)):
        price_cmc[d, :] = heston_cmc_qe.price(strike, forward, texp, sigma=sigma, delta=delta[d], intr=r, path=1e6, seed=123456)
        bias_cmc[d, :] = price_cmc[d, :] - ref_price[i, :]

    end = time.time()
    
    print('Case %s:' % i)
    np.set_printoptions(suppress=True)
    print('computed price')
    print(price_cmc)
    
    print('bias')
    print(bias_cmc)
    print('Running time is %.3f seconds.' % (end - start) + '\n')

Case 0:
computed price
[[14.52988166  0.1954481  45.5566836 ]
 [13.58264697  0.26538629 44.59576156]
 [13.18875382  0.29164219 44.35693654]
 [13.09245767  0.29643382 44.32630416]
 [13.09234981  0.29612097 44.3554616 ]
 [13.0747208   0.29571157 44.30422413]]
bias
[[ 1.44488166 -0.1005519   1.2266836 ]
 [ 0.49764697 -0.03061371  0.26576156]
 [ 0.10375382 -0.00435781  0.02693654]
 [ 0.00745767  0.00043382 -0.00369584]
 [ 0.00734981  0.00012097  0.0254616 ]
 [-0.0102792  -0.00028843 -0.02577587]]
Running time is 1123.970 seconds.

Case 1:
computed price
[[33.73639816 17.45658862 56.93686071]
 [33.6862614  17.7939801  56.72279494]
 [33.62950685 18.06468289 56.61902551]
 [33.60258667 18.14381343 56.57858711]
 [33.59659541 18.15261599 56.57703292]
 [33.59741061 18.15529961 56.5780489 ]]
bias
[[ 0.13939816 -0.70041138  0.36186071]
 [ 0.0892614  -0.3630199   0.14779494]
 [ 0.03250685 -0.09231711  0.04402551]
 [ 0.00558667 -0.01318657  0.00358711]
 [-0.00040459 -0.00438401  0.00203292]
 [ 0.0004

KeyboardInterrupt: 

In [ ]:
# compute std error of conditional mc price
n = 50
for i in range(case.shape[0]):
    start = time.time()
    vov, kappa, rho, texp, theta, sigma, r = case[i]

    heston_cmc_qe = heston.HestonCondMcQE(vov=vov, kappa=kappa, rho=rho, theta=theta)
    price_cmc = np.zeros([len(delta), len(strike), n])
    for j in tqdm(range(n)):
        for d in range(len(delta)):
            price_cmc[d, :, j] = heston_cmc_qe.price(strike, forward, texp, sigma=sigma, delta=delta[d], intr=r, path=1e4)
            
    end = time.time()
    print('Case %s:' % i)
    print('computed price')
    print(price_cmc.mean(axis=2))
    print('std error of conditional MC')
    print(price_cmc.std(axis=2))
    print('Running time is %.3f seconds.' % (end - start) + '\n')

In [ ]:
# compare our model with:
# a.conditional MC with TG scheme
# b.exact MC
# c.almost exact MC


## Von's (2008) example, with multiple spots and single strike

In [ ]:
# Example
# Von (2018), multiple forward, single strike
strike = [100.0]
forward = [75, 100, 125]
delta = [1, 1/2, 1/4, 1/8, 1/16, 1/32]
case = np.zeros([3, 7])
vov = 1
kappa = 2.58
rho = -0.36
texp = 1
theta = 0.043
sigma = np.sqrt(0.114)
r = 0
ref_price = np.array([0.908502728459621, 9.046650119220969, 28.514786399298796])

In [ ]:
# compute conditional mc price and bias
# for cases with single strike and multiple forward
price_cmc = np.zeros([len(delta), len(forward)])
bias_cmc = np.zeros([len(delta), len(forward)])
start = time.time()
for i in range(len(forward)):
    heston_cmc_qe = heston.HestonCondMcQE(vov=vov, kappa=kappa, rho=rho, theta=theta)
    
    for d in range(len(delta)):
        price_cmc[d, i] = heston_cmc_qe.price(strike, forward[i], texp, sigma=sigma, delta=delta[d], intr=r, path=1e6, seed=123456)
        bias_cmc[d, i] = price_cmc[d, i] - ref_price[i]

end = time.time()
np.set_printoptions(suppress=True)
print('price:')
print(price_cmc)
print('bias:')
print(bias_cmc)
print('Running time is %.3f seconds.' % (end - start) + '\n')